In [11]:
from pandas import concat,DataFrame
import numpy as np 
import pandas as pd


In [12]:
df = pd.read_csv("city_hour.csv") 
df=df[df['City']=='Chennai']
df.drop(columns=['City','NOx','PM10','SO2','NO','AQI_Bucket','Benzene','Toluene','Xylene'],axis=1,inplace=True)
df.head(5)


,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
187751,2015-01-01 01:00:00,NaN,47.97,6.36,0.92,7.65,NaN
187752,2015-01-01 02:00:00,NaN,12.31,4.93,0.77,10.96,NaN
187753,2015-01-01 03:00:00,NaN,13.72,3.92,1.06,12.14,NaN
187754,2015-01-01 04:00:00,NaN,14.45,4.54,1.54,11.95,NaN
187755,2015-01-01 05:00:00,NaN,20.16,4.85,1.03,11.64,NaN


In [13]:
df.interpolate(limit_direction="both",inplace=True)
df.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
187751,2015-01-01 01:00:00,428.3,47.97,6.36,0.92,7.65,333.0
187752,2015-01-01 02:00:00,428.3,12.31,4.93,0.77,10.96,333.0
187753,2015-01-01 03:00:00,428.3,13.72,3.92,1.06,12.14,333.0
187754,2015-01-01 04:00:00,428.3,14.45,4.54,1.54,11.95,333.0
187755,2015-01-01 05:00:00,428.3,20.16,4.85,1.03,11.64,333.0


In [6]:
e = pd.DataFrame(df[['PM2.5','NO2','NH3','O3','CO']].values)

In [7]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(list(df['AQI']),exog=e,order=(1,0,1),seasonal_order=(0,0,0,24),enf48orce_invertibility=False)
results = model.fit()

C:\Users\ssneh\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [8]:
exog_forecast = e[48187:].values
fcast = results.predict(len(df),len(df)+4,exog=exog_forecast)

In [9]:
y = fcast.values
y = y.ravel()
print("Forecasted AQI:")
print(y)

Forecasted AQI:
[91.67701061 91.47296418 90.85415775 90.67062211 89.84753767]


In [10]:
from sklearn.metrics import mean_squared_error
from math import sqrt

df1=df['AQI']
actual_values=df1[-5:]
#calculate RMSE
r = sqrt(mean_squared_error(actual_values, y)) 
print("RMSE:")
print(r)

RMSE:
2.147246125339176
